# canopy components by time
breakdown one run of a experiment into its canopy componenets to figure out which are heavy

In [1]:
# import  epmt query 
print('importing epmt_query')
import epmt_query as eq
# import matplot for better plotting functions
import sys
sys.path.insert(0,'/home/Ian.Laflotte/ians_py374/pip_experiment')
import matplotlib.pyplot as plt
import numpy as np
# import pandas. optional but helpful 'display.max_columns' arg shows all DataFrame columns when printing
print('importing pandas')
import pandas
pandas.set_option('display.max_columns', None)

importing epmt_query
importing pandas


In [2]:
import epmt_query as eq

def main(exp_name):
    job_type_list=['analysis', 'clean', 'combine', 'data-catalog', 'make-timeavgs','mask-atmos-plevel','pp-starter','regrid-xy','remap-pp-components', 'rename-split-to-pp','split-netcdf','stage-history']
    canopy_jobs=eq.get_jobs( fltr=(eq.Job.user_id == 'oar.gfdl.am5'), tags = 'exp_fre_mod:canopy;exp_name:'+exp_name, fmt = 'dict')
    
    
    job_type_ids_dict={}
    for job_type in job_type_list:
        job_type_ids_dict[job_type]=[]
        
    print(job_type_ids_dict)
        
    for job in canopy_jobs:
        jobname=job['jobname']
        for job_type in job_type_list:
            if job_type in jobname:
                job_type_ids_dict[job_type].append(job['jobid'])
                break
    
    return job_type_ids_dict
            


In [18]:
exp_name = 'c96L65_am5f5b7r0_pdclim2010F'
canopy_ids = main(exp_name) 
keys = ['analysis', 'clean', 'combine', 'data-catalog', 'make-timeavgs','mask-atmos-plevel','pp-starter','regrid-xy','remap-pp-components', 'rename-split-to-pp','split-netcdf','stage-history']

{'analysis': [], 'clean': [], 'combine': [], 'data-catalog': [], 'make-timeavgs': [], 'mask-atmos-plevel': [], 'pp-starter': [], 'regrid-xy': [], 'remap-pp-components': [], 'rename-split-to-pp': [], 'split-netcdf': [], 'stage-history': []}


In [19]:
#sort all the data based on canopy or bronx
for key in keys:
    exp_jobs = eq.get_jobs(jobs = canopy_ids[key])
    metrics_list = ['rssmax', 'num_procs', 'duration', 'write_bytes'] #designed for 4 metrics
    bronx_jobs, canopy_jobs = [],[]
    bronx_metrics,canopy_metrics = {}, {}
    canopy_time, bronx_time = 0,0 
    canopy_duration, bronx_duration = 0,0 

    for metrics_key in metrics_list:  #set keys for dictionaries
        bronx_metrics[metrics_key] = []
        canopy_metrics[metrics_key] = []
    #loop through once, and sort jobs into categories
    for job in exp_jobs:
        if job['tags']['exp_fre_mod'].find('canopy') >= 0:
            canopy_jobs.append(job)
            for metrics_key in metrics_list:
                canopy_metrics[metrics_key].append(job[metrics_key])
                canopy_time += job['cpu_time'] +job['time_waiting']
                canopy_duration += job['duration']
    #final calculations     
    counts = [len(bronx_jobs), len(canopy_jobs)]        
    averages = {'rssmax':[0,0],'num_procs':[0,0],'duration':[0,0],'write_bytes':[0,0]} #always ['bronx','canopy']
    errors = {'rssmax':[0,0],'num_procs':[0,0],'duration':[0,0],'write_bytes':[0,0]}
    for metrics_key in metrics_list:
        averages[metrics_key][1] = sum(canopy_metrics[metrics_key])/len(canopy_jobs)
        errors[metrics_key][1] = np.std(canopy_metrics[metrics_key])/(len(canopy_jobs))**.5
    #print the info
    print('The total time for experiment ', key, exp_name, ' is ', canopy_time/1e9, ' seconds with canopy. Duration is ',canopy_duration/1e9,' seconds')



The total time for experiment  analysis c96L65_am5f5b7r0_pdclim2010F  is  9765.975646584  seconds with canopy. Duration is  660.807145424  seconds
The total time for experiment  clean c96L65_am5f5b7r0_pdclim2010F  is  289.456589328  seconds with canopy. Duration is  5.659381968  seconds
The total time for experiment  combine c96L65_am5f5b7r0_pdclim2010F  is  803.25413218  seconds with canopy. Duration is  46.341223128  seconds
The total time for experiment  data-catalog c96L65_am5f5b7r0_pdclim2010F  is  90.896683476  seconds with canopy. Duration is  1.03300802  seconds
The total time for experiment  make-timeavgs c96L65_am5f5b7r0_pdclim2010F  is  2507.507931068  seconds with canopy. Duration is  88.000034908  seconds
The total time for experiment  mask-atmos-plevel c96L65_am5f5b7r0_pdclim2010F  is  684.381794204  seconds with canopy. Duration is  26.0961268  seconds
The total time for experiment  pp-starter c96L65_am5f5b7r0_pdclim2010F  is  76.88919542  seconds with canopy. Duration i